<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2020m0/9%20Preparation/Redlich_Kwong_CP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## setting up colab to pretty-print sympy formulae
from sympy          import init_printing
from sympy.printing import latex

def colab_LaTeX_printer(exp, **options):  
    from google.colab.output._publish import javascript 

    url_ = "https://colab.research.google.com/static/mathjax/MathJax.js?"
    cfg_ = "config=TeX-MML-AM_HTMLorMML" # "config=default"

    javascript(url=url_+cfg_)

    return latex(exp, **options)
# end of def
init_printing(use_latex="mathjax", latex_printer=colab_LaTeX_printer) 


## Redlich-Kwong状態方程式の臨界点


状態方程式の変曲点は、熱力学では臨界点と呼びます。体積変化に対して圧力が変動しないということは、圧縮率が無限大になり、体積が大きくゆらぐことを示唆しています。圧縮率が無限大ということはまた、音速が0になることを意味します。

The inflection point of the state equation is called the critical point in thermodynamics.
As we see above, $\partial p/\partial V$ becomes zero, i.e.,  the compressibility $\partial V/\partial p$ becomes infinite.
At the critical point, the volume fluctuates greatly, and the sound velocity becomes zero.


In [3]:
from sympy import *
from sympy.abc import *

# RK状態方程式
p=k*T/(V-b)-a/(sqrt(T)*V*(V+b))
# 微分と二階微分が同時に0になるのが臨界点。
p1=p.diff(V)
p2=p.diff(V,2)

In [4]:
# 分数を避ける。
pp1 = (p1*(V*(V-b)*(V+b))**2*sqrt(T)).simplify()
pp1

   3/2  2          2                2              2
- T   ⋅V ⋅k⋅(V + b)  + 2⋅V⋅a⋅(V - b)  + a⋅b⋅(V - b) 

In [5]:
pp2 = (p2*(V*(V-b)*(V+b))**3*sqrt(T)).simplify()
pp2

   3/2  3          3      2          3                  3              3      
2⋅T   ⋅V ⋅k⋅(V + b)  - 4⋅V ⋅a⋅(V - b)  - 2⋅V⋅a⋅b⋅(V - b)  - 2⋅a⋅(V - b) ⋅(V + 

  2
b) 

V,Tの解は計算できない(無限ループ?)ので、(a,b)を求めて、あとで式変形することにしよう。

In [6]:
sol = solve([pp1,pp2],(a,b))
sol

⎡         ⎛ 3/2     ⎛    3 ___    2/3⎞                ⎞  ⎛ 3/2     ⎛   2/3   3
⎢         ⎜T   ⋅V⋅k⋅⎝1 + ╲╱ 2  + 2   ⎠    ⎛     3 ___⎞⎟  ⎜T   ⋅V⋅k⋅⎝- 2    - ╲
⎢(0, -V), ⎜───────────────────────────, V⋅⎝-1 + ╲╱ 2 ⎠⎟, ⎜────────────────────
⎣         ⎝             3                             ⎠  ⎝                    

 ___       3 ___         2/3     ⎞                             ⎞  ⎛ 3/2     ⎛ 
╱ 2  + 2 - ╲╱ 2 ⋅√3⋅ⅈ + 2   ⋅√3⋅ⅈ⎠    ⎛     3 ___ ⎛  1   √3⋅ⅈ⎞⎞⎟  ⎜T   ⋅V⋅k⋅⎝-
──────────────────────────────────, V⋅⎜-1 + ╲╱ 2 ⋅⎜- ─ - ────⎟⎟⎟, ⎜───────────
      6                               ⎝           ⎝  2    2  ⎠⎠⎠  ⎝           

  2/3   3 ___        2/3        3 ___     ⎞                             ⎞⎤
 2    - ╲╱ 2  + 2 - 2   ⋅√3⋅ⅈ + ╲╱ 2 ⋅√3⋅ⅈ⎠    ⎛     3 ___ ⎛  1   √3⋅ⅈ⎞⎞⎟⎥
───────────────────────────────────────────, V⋅⎜-1 + ╲╱ 2 ⋅⎜- ─ + ────⎟⎟⎟⎥
               6                               ⎝           ⎝  2    2  ⎠⎠⎠⎦

4つの根のうち、実数でかつ意味があるのは1つだけ。sol[1]の1つめがa, 2つめがbである。

In [7]:
sol[1]

⎛ 3/2     ⎛    3 ___    2/3⎞                ⎞
⎜T   ⋅V⋅k⋅⎝1 + ╲╱ 2  + 2   ⎠    ⎛     3 ___⎞⎟
⎜───────────────────────────, V⋅⎝-1 + ╲╱ 2 ⎠⎟
⎝             3                             ⎠

bの式を変形すれば、臨界体積Vcが得られる。

In [8]:
Vc = solve(sol[1][1]-b, V)[0]
Vc

    b     
──────────
     3 ___
-1 + ╲╱ 2 

In [17]:
Vc=b/(-1+2**(1/3))
Vc

3.84732210186307⋅b

aの式をTの式とし、体積を臨界体積におきかえると、臨界温度Tcがaとbで表せる。

In [9]:
Tc=solve(sol[1][0]-a,T)[0].subs(V,Vc).simplify()
Tc

          2/3             2/3
 2/3 ⎛ a ⎞    ⎛     3 ___⎞   
3   ⋅⎜───⎟   ⋅⎝-1 + ╲╱ 2 ⎠   
     ⎝b⋅k⎠                   
─────────────────────────────
                      2/3    
    ⎛    3 ___    2/3⎞       
    ⎝1 + ╲╱ 2  + 2   ⎠       

In [10]:
3.0**(2/3)*(-1+2**(1/3))**(2/3)/(1+2**(1/3)+2**(2/3))**(2/3)

0.34503999585334744

In [13]:
Tc=0.34503999585334744*(a/(b*k))**Rational(2,3)
Tc

                       2/3
                  ⎛ a ⎞   
0.345039995853347⋅⎜───⎟   
                  ⎝b⋅k⎠   

さいごに、pの式(状態方程式)のT,VをそれぞれTc,Vcにするとpcが得られる。

In [18]:
pc=p.subs(V,Vc).subs(T,Tc).simplify()
pc

0.0298943859926416⋅a
────────────────────
           _____    
     2    ╱  a      
    b ⋅3 ╱  ───     
       ╲╱   b⋅k     

[Wikipedia:Redlich-Kwong](https://en.wikipedia.org/wiki/Redlich%E2%80%93Kwong_equation_of_state)に一致したように思える。

In [1]:
# Wikipediaの式の係数を数値計算して検算
#pcの係数
(2**(1/3)-1)**(7/3)/3**(1/3)

0.02989438599264158

In [2]:
#Tcの係数
3**(2/3)*(2**(1/3)-1)**(4/3)

0.34503999585334744

あってました。